# PyUnfold: The Python Unfolding Package

## Zigfried Hampel-Arias

### IIHE -- Brussels, BE
23 May, 2018

### PyUnfold Site:
https://jrbourbeau.github.io/pyunfold/index.html <br>

### PyUnfold GitHub repo:
https://github.com/jrbourbeau/pyunfold <br>

### Slides:
https://zhampel.github.io/intro-pyunfold-iihe


### Contact:
E-mail: [jbourbeau@icecube.wisc.edu](mailto:jbourbeau@icecube.wisc.edu), [zhampel@wipac.wisc.edu](mailto:zhampel@wipac.wisc.edu)

<a href="https://zhampel.github.io/">
<img src="images/octocat.png" alt="Go My GitHub" width="60" height="60" border="0"> </a>

<a href="https://www.linkedin.com/in/zhampel-arias/">
<img src="images/LinkedIn-InBug-2C.png" alt="Go to my LinkedIn" width="60" height="60" border="0">
</a>

# The Measurement Process

An *ideal* detector:
- Makes **no error** measuring a desired quantity
- Makes **no bias** or shift in a desired quantity

Real world detectors have:
- Finite resolutions
- Characteristic biases
- Efficiencies <100%
- Statistical + systematic uncertainties

# Measuring Distributions


- Distribution $\phi(C)$ of **causes** $C_{\mu}$
- Distribution $n(E)$ of **effects** $E_j$
- Detector *response matrix* $R_{\mu j}$ connecting **causes &rarr; effects**

$$
n(E) = \mathbf{R} \, \phi(C)
$$

- $n(E)$ is what we **observe**
- $\mathbf{R}$ is known or estimated
- $\phi(C)$ is the truth what we **want**

# Physics Example: Cosmic Ray Spectrum

| ![effects](images/effects.png)  | ![response](images/response-matrix.png) | <img src="images/causes.jpeg" width="700"> |
|:---:|:---:|:---:|
| $n(E)$ | $\mathbf{R}$ | $\phi(C)$ |

From [Phys. Rev. D 96, 122001](https://journals.aps.org/prd/abstract/10.1103/PhysRevD.96.122001)

# Getting What We Want

Two ways to get $\phi(C)$:

- Guess a form of $\phi(C)$ and *fold* with response $\mathbf{R}$
    - Easy, *cheap* to evaluate
    - **Restricted** to analytic forms

- Start with $n(E)$ and *unfold* $\phi(C)$
    - **No restrictions** on form
    - Have to *trust* solution


# Cosmic Rays Again

| ![effects](images/effects.png)  | ![response](images/response-matrix.png) | <img src="images/causes.jpeg" width="700"> |
|:---:|:---:|:---:|
| $n(E)$ | $\mathbf{R}$ | $\phi(C)$ |
| **Unfolding** &rarr;|  | &larr; **Folding** |

From [Phys. Rev. D 96, 122001](https://journals.aps.org/prd/abstract/10.1103/PhysRevD.96.122001)

# Deconvolution

So why don't we just invert $\mathbf{R}$?

What if
- an analytic form of $\mathbf{R}$ d.n.e.?
- $\mathbf{R}$ is built from finite simulation?
- $\mathbf{R}$ is not be invertible?

Proposal: build a matrix $\mathbf{M}$ s.t.

$$
\begin{align}
    \mathbf{M} &\approx \mathbf{R}^{-1} \\
    \phi(C) &\approx \mathbf{M} \, n(E)
\end{align}
$$

# PyUnfold

![pyunfold](images/logo-dark.png)

A Python package to account for imperfect measurements in an iterative *unfolding*.

# PyUnfold

A Python package to account for imperfect measurements in an iterative *unfolding*\*.

Authors: [James Bourbeau](mailto:jbourbeau@icecube.wisc.edu), [Zig Hampel](mailto:zhampel@wipac.wisc.edu)

PyUnfold provides users:
- A tool for the analysis of measurement effects on physical distributions, i.e. calculate $\mathbf{M}$
- A straightforward API that's **experimentally agnostic**, beyond HEP
- The ability to **easily** incorporate both $\sigma_{\text{stat}}$ and $\sigma_{\text{sys}}$

Installation: `pip install pyunfold`

\* [D'Agostini (1995)](https://www.sciencedirect.com/science/article/pii/016890029500274X)

## PyUnfold Inputs

To use PyUnfold, one needs only to provide the

- Measured effects distribution
- Response matrix
- Prior distribution

In [1]:
    from pyunfold import iterative_unfold

    # Observed distributions
    data = [100, 150]
    data_err = [10, 12.2]

    # Response matrix
    response = [[0.9, 0.1],
                [0.1, 0.9]]
    response_err = [[0.01, 0.01],
                    [0.01, 0.01]]

    # Detection efficiencies
    efficiencies = [1, 1]
    efficiencies_err = [0.01, 0.01]

In [2]:
# Perform iterative unfolding
unfolded_result = iterative_unfold(data=data,
                                   data_err=data_err,
                                   response=response,
                                   response_err=response_err,
                                   efficiencies=efficiencies,
                                   efficiencies_err=efficiencies_err)

## Features: Prior

Flexibility of prior to test results's robustness
   - Uniform
   - Jeffreys
   - User-defined
    
Other unfolding toolkits do not permit user defined prior.

In [3]:
from pyunfold import iterative_unfold
from pyunfold.callbacks import Logger

import numpy as np
np.random.seed(2)

# True distribution
num_samples = int(1e5)
true_samples = np.random.normal(loc=10.0, scale=4.0, size=num_samples)
bins = np.linspace(0, 20, 21)
num_causes = len(bins) - 1
data_true, _ = np.histogram(true_samples, bins=bins)

# Observed distribution
random_noise = np.random.normal(loc=0.3, scale=0.5, size=num_samples)
observed_samples = true_samples + random_noise
data_observed, _ = np.histogram(observed_samples, bins=bins)
data_observed_err = np.sqrt(data_observed)

# Efficiencies
efficiencies = np.ones_like(data_observed, dtype=float)
efficiencies_err = np.full_like(efficiencies, 0.1, dtype=float)

# Response matrix
response_hist, _, _ = np.histogram2d(observed_samples, true_samples, bins=bins)
response_hist_err = np.sqrt(response_hist)

# Normalized response
column_sums = response_hist.sum(axis=0)
normalization_factor = efficiencies / column_sums

response = response_hist * normalization_factor
response_err = response_hist_err * normalization_factor

In [4]:
from pyunfold.priors import jeffreys_prior, uniform_prior
cause_lim = np.logspace(0, 3, num_causes)
uni_prior = uniform_prior(num_causes)
jeff_prior = jeffreys_prior(cause_lim)

## Features: Prior

![priors](images/priors.png)

In [5]:
print('Running with uniform prior...')
unfolded_uniform = iterative_unfold(data=data_observed,
                                    data_err=data_observed_err,
                                    response=response,
                                    response_err=response_err,
                                    efficiencies=efficiencies,
                                    efficiencies_err=efficiencies_err,
                                    ts='ks',
                                    ts_stopping=0.01,
                                    callbacks=[Logger()])

print('\nRunning with Jeffreys prior...')
unfolded_jeffreys = iterative_unfold(data=data_observed,
                                     data_err=data_observed_err,
                                     response=response,
                                     response_err=response_err,
                                     efficiencies=efficiencies,
                                     efficiencies_err=efficiencies_err,
                                     prior=jeff_prior,
                                     ts='ks',
                                     ts_stopping=0.01,
                                     callbacks=[Logger()])

Running with uniform prior...
Iteration 1: ts = 0.1460, ts_stopping = 0.01
Iteration 2: ts = 0.0060, ts_stopping = 0.01

Running with Jeffreys prior...
Iteration 1: ts = 0.7231, ts_stopping = 0.01
Iteration 2: ts = 0.0171, ts_stopping = 0.01
Iteration 3: ts = 0.0006, ts_stopping = 0.01


![prior-unfolded](images/prior-unfolded.png)

## Features: Convergence

Stopping criteria based on test statistic (TS)
   - User defined TS tolerance
   - K-S, $\chi^2$, RMD, Bayes factor
   
Other unfolding toolkits set default $N_{\text{iter}}=4$. 

M. [Kuusela](https://indico.cern.ch/event/671301/contributions/2746051/attachments/1557280/2449596/DIANA_Nov_2017.pdf): Countless LHC papers using $4$ iterations of D'Agostini unfolding.

## Features: Regularization

- Smoothing at each iteration with univariate spline
    - Optional callback
    - Strength parameter is required variable
- **Multidimensional** capabilities
    - Groups or blocks of causes
    - $R_{\mu j} \to R_{\mu \nu j}$ *unravelled*
    - Regularization **only** in groups
    
Optimal reg. strength problem dependent, **default does not exist**.

## Regularizing a Bad Prior

![bumpy-prior](images/bumpy-prior.png)

## Regularizing a Bad Prior

![regularization](images/regularization.png)

# Extensive Documentation

### PyUnfold Site:
https://jrbourbeau.github.io/pyunfold/index.html <br>

### PyUnfold GitHub repo:
https://github.com/jrbourbeau/pyunfold <br>


### Other:
[Introductory](https://jrbourbeau.github.io/pyunfold/notebooks/tutorial.html) and 
[advanced](https://jrbourbeau.github.io/pyunfold/advanced.html) example notebooks

Mathematical [details](https://jrbourbeau.github.io/pyunfold/mathematics.html)


## Other Unfolding Resources

D'Agostini:
- Original [paper](https://www.sciencedirect.com/science/article/pii/016890029500274X)
- [Update](https://arxiv.org/abs/1010.0632)

Statistician M. Kuusela:

- [Intro to Unfolding Methods](https://mkuusela.web.cern.ch/mkuusela/T2K_workshop_Sep_2016/T2K_unfolding_Sep_2016.pdf)
- [Current View of Unfolding Software](https://indico.cern.ch/event/671301/contributions/2746051/attachments/1557280/2449596/DIANA_Nov_2017.pdf)

# Current Status

- Can be used used out of the box right now: `pip install pyunfold`

- Flexible framework. [Submit](https://jrbourbeau.github.io/pyunfold/contributing.html) an issue if you see something you want to change or see!

- PyUnfold submitted to J.O.S.S. Under [review](https://github.com/openjournals/joss-papers/blob/joss.00741/joss.00741/10.21105.joss.00741.pdf)

- Designed to make it easier to:
    - run an iterative unfolding
    - test robustness of results

- Hoping to make unfolding accessible beyond HEP

# Thank you!
![sunset](images/sunset-gsl.jpg)